In [1]:
import configparser

config = configparser.ConfigParser()
config.read(r'C:\Users\GisselleMaira\test.ini')

import json
import os
import urllib.request
import requests
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime, timedelta
from selenium import webdriver
import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.relative_locator import locate_with
from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
import shutil
import glob
import os.path
import os
import time
import configparser
import win32com.client
from dateutil.relativedelta import relativedelta
import xlrd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import holidays

In [2]:
today = date.today()
prev_4month = today - relativedelta(months=4)
prev_4month = prev_4month.replace(day=1)


#get dynamic date for last 3 months
today = date.today()

tgt = datetime.datetime(2021, 1, 1)
d1 = tgt.strftime("%Y-%m-%d")
d2 = prev_4month
# curr_month = today.replace(day=1)
# d2 = curr_month.strftime("%Y-%m-%d")

start = today + timedelta(days=-today.weekday(), weeks=-18)



acct = config['api']['account']
token = config['api']['token']

print(d1, d2)

2021-01-01 2021-12-01


In [3]:
payload={}
headers = {
  'Harvest-Account-Id': acct,
  'Authorization': token
}

## Pull invoice data

In [4]:
url = f"https://api.harvestapp.com/v2/invoices?from={d1}&page=1"

invoices = []

while url:
    print('Requesting', url)
    
    response = requests.request("GET", url, headers=headers, data=payload).text
    data_json  = json.loads(response)['invoices']
    invoices.extend(data_json)
    
    url = json.loads(response)['links']['next']
    

Requesting https://api.harvestapp.com/v2/invoices?from=2021-01-01&page=1
Requesting https://api.harvestapp.com/v2/invoices?from=2021-01-01&page=2&per_page=100&ref=next


In [5]:
items = pd.json_normalize(invoices, meta = ['number'], record_path = ['line_items'])
totals = pd.json_normalize(invoices)

In [6]:
invoice_df = pd.merge(totals, items, on = 'number', how= 'left')

invoice_df.columns = invoice_df.columns.str.replace('.', '_')

In [7]:
invoice_df.rename(columns = {'number': 'invoice_num', 'kind': 'services_provided', 'amount_y': 'ttl_item_amount', \
                  'amount_x': 'invoice_amt', 'id_x': 'invoice_id'}, inplace=True)
invoice_df.drop(columns=['client_key', 'purchase_order', 'tax', 'tax_amount', 'tax2', 'tax2_amount',\
                        'discount', 'notes', 'recurring_invoice_id', 'created_at', 'updated_at',  'currency',\
                         'estimate', 'retainer', 'id_y', 'paid_at', 'creator_name', 'creator_id', 'taxed', \
                         'taxed2', 'line_items', 'project'], axis=1, inplace = True)

# summary table

# invoice_df[['client_name', 'project_name', 'project_code', 'period_end', 'subject', 'invoice_num', \
#             'invoice_amt', 'state']].drop_duplicates()
# invoice_df.to_excel(r'C:\Users\GisselleMaira\Desktop\invoices.xlsx', index = None, header=True)

In [8]:
invoice_df['user_name'] = invoice_df['description'].str.split(':', expand=True)[1].str.split('(', expand=True)[0]
invoice_df.rename(columns = {'unit_price': 'hourly_rate'}, inplace = True)

rate_card = invoice_df.drop_duplicates(subset=['user_name', 'project_name', 'hourly_rate'] )[['user_name', 'project_name', 'hourly_rate']]
rate_card = rate_card.dropna(axis=0, subset=['project_name', 'user_name']).sort_values(['project_name', 'user_name']).reset_index()

rate_card.drop('index',1, inplace=True)

In [9]:
rate_card.user_name = rate_card.user_name.str.strip()
rate_card = rate_card.loc[~rate_card.user_name.isin (['Credit Strategy Assessment', 'Client work', 'RDAR Reporting', 'Discount for hours concession represented above'])]
rate_card = rate_card.loc[~rate_card.project_name.isin (['Senior Leadership Training'])]


## Pull time entries (recent only) and assignments

In [10]:
url = f"https://api.harvestapp.com/v2/time_entries?from={d2}&per_page=100&page=1"
all_data = []

while url:
    print('Requesting', url)
    
    response = requests.request("GET", url, headers=headers, data=payload).text
    data_json  = json.loads(response)['time_entries']
    all_data.extend(data_json)
    
    url = json.loads(response)['links']['next']

Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&per_page=100&page=1
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=2&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=3&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=4&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=5&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=6&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=7&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=8&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=9&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/time_entries?from=2021-12-01&page=10&per_page=100&ref=next
Requesting https:/

In [11]:
item_df = pd.json_normalize(all_data,max_level=1)
item_df.columns = item_df.columns.str.replace('.', '_')
# item_df.columns

In [12]:
url_assign = f"https://api.harvestapp.com/v2/user_assignments?page=1&per_page=100"
all_data_assign = []

while url_assign:
    print('Requesting', url_assign)
    
    response_assign = requests.request("GET", url_assign, headers=headers, data=payload).text
    data_json_assign  = json.loads(response_assign)['user_assignments']
    all_data_assign.extend(data_json_assign)
    
    url_assign = json.loads(response_assign)['links']['next']

Requesting https://api.harvestapp.com/v2/user_assignments?page=1&per_page=100
Requesting https://api.harvestapp.com/v2/user_assignments?page=2&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=3&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=4&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=5&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=6&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=7&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=8&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=9&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=10&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assignments?page=11&per_page=100&ref=next
Requesting https://api.harvestapp.com/v2/user_assi

In [13]:
assign_df = pd.json_normalize(all_data_assign,max_level=1)
assign_df.columns = assign_df.columns.str.replace('.', '_')

In [14]:
assign_df = assign_df[['project_id', 'project_name', 'project_code', 'user_id', 'user_name', 'hourly_rate']]
item_df = item_df[['spent_date','user_id', 'user_name','client_id', 'client_name', 'rounded_hours', 'project_id','project_name',\
            'project_code', 'task_id', 'task_name', 'notes', 'is_closed', 'is_billed', 'billable',
            'budgeted', 'invoice']]

In [15]:
assign_merge = pd.merge(item_df, assign_df, on=['user_id', 'user_name','project_id', 'project_name', 'project_code'], how='left')

## get user roles and emails

In [16]:
URL_users = f"https://api.harvestapp.com/v2/users?page=1&per_page=100"
all_users = []

while URL_users:
    print('Requesting', URL_users)
    
    response_users = requests.request("GET", URL_users, headers=headers, data=payload).text
    data_json_users  = json.loads(response_users)['users']
    all_users.extend(data_json_users)
    
    URL_users = json.loads(response_users)['links']['next']

Requesting https://api.harvestapp.com/v2/users?page=1&per_page=100
Requesting https://api.harvestapp.com/v2/users?page=2&per_page=100&ref=next


In [17]:
users = pd.json_normalize(all_users,max_level=1)
users2 = users.explode('roles')[['id', 'first_name', 'last_name','is_project_manager', 'is_active', 'default_hourly_rate', 'cost_rate', 'roles']]

# isolate emails and filter out FP - they can have multiple assignments
# FPemail = users2[users2['roles'].isin(['Flying Phase'])]['email']
# nonFP = users2[~users2.email.isin(FPemail)]
# FP = users2[users2.email.isin(FPemail)]

# get advisor/consultants/ principals.
# FP2 = FP[FP['roles'].isin(['Advisor', 'Consultant', 'Principal'])][['id', 'first_name', 'last_name', 'email', 'default_hourly_rate', 'cost_rate', 'roles']]
# BU2 = users2[users2['roles'].isin(['Flying Phase'])][['id', 'email', 'roles']]
# FP3 = pd.merge(FP2, BU2 , on = ['id', 'email'], how='left')

# nonFP2 = nonFP[nonFP['roles'].isin(['Advisor', 'Consultant', 'Principal'])][['id', 'first_name', 'last_name', 'email', 'default_hourly_rate', 'cost_rate', 'roles']]
# BU1 = users2[users2['roles'].isin(['CCOM', 'Risk', 'Data and Analytics'])][['id', 'email', 'roles']]
# nonFP3 = pd.merge(nonFP2, BU1 , on = ['id', 'email'], how='left')

users2A = users2[users2['roles'].isin(['Advisor', 'Consultant', 'Principal'])]
users2B = users2[users2['roles'].isin(['Risk', 'Flying Phase', 'CCOM', 'Data and Analytics'])][['id', 'roles']]
all_users2 = pd.merge(users2A, users2B , on = ['id'], how='left')

all_users2.rename(columns={'roles_x': 'role', 'roles_y': 'BusUnit'}, inplace=True)

In [18]:
users3 = all_users2[[ 'id', 'role', 'BusUnit', 'first_name', 'last_name']] 
users3.rename({'id':'user_id'}, axis=1, inplace=True)

users3['user_name'] = users3['first_name'] + ' ' + users3['last_name']
users3.drop(['first_name', 'last_name'],1,inplace = True)

C:\Users\GisselleMaira\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-18-c7e454aabada>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users3['user_name'] = users3['first_name'] + ' ' + users3['last_name']
C:\Users\GisselleMaira\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [19]:
users3.loc[users3['user_name'].isin([
    'Brett Ludden', 'Adam Searles', 'Parag Patwardhan', 'Kim Poremski',
    'Kevin Baquero', 'Parker Barouch', 'Jim Euchner', 'Shekhar Nagendra'
]), ['BusUnit']] = 'Data and Analytics'


users3.loc[users3['user_name'].isin(['Christine Dodson', 'Lena Sapia', 'Anthony Gibbs']),
              ['BusUnit']] = 'Risk'

users3.loc[users3['user_name'].
              isin(['Keith Schleicher', 'Sunil Kumar Shambulingaiah', 'Chris Landrum', 'Dana Bennett',
                   'Mark Schuler', 'Kristina Schneider', 'Kenny Martin', 'Erin Clark', 'Jeff Nelson',
                   'Stuart Williams', 'Greg Marcel', 'Robert Johnson', 'Rachelle Peay', 'Micah Dalton',
                   'Brian Eugley', 'Daniel Shao', 'Matt Borgard', 'Victoria Dlugosz', 'Jeri Mulholland', 
                   'Terrell Sanders', 'Jeff Brantley', 'Jaime Guzman', 'Samantha Bourne', 'Dave Critchley',
                   'Philip Paige', 'Sunil Shambulingaiah']),
              ['BusUnit']] = 'Flying Phase'

users3.loc[users3['user_name'].isin(
    ['Jeff Marshall', 'Scott Hamilton', 'Scott Summers', 'Deb Hollowell', 'Aloma Holsten', 'Ken Roberts']),
              ['BusUnit']] = 'CCOM'

# users3[users3['BusUnit'].isnull()].sort_values(by=['user_name'])
# users3[users3['BusUnit'].isnull()]

C:\Users\GisselleMaira\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [20]:
merge_all = pd.merge(assign_merge, users3, on = ['user_id', 'user_name'], how='left')
merge_all['spent_date'] = pd.to_datetime(merge_all['spent_date']) 
merge_all['wk_start'] = merge_all['spent_date'] - pd.TimedeltaIndex(merge_all.spent_date.dt.dayofweek,unit='d') 

merge_all['month'] = merge_all['spent_date'].to_numpy().astype('datetime64[M]')

In [21]:
invoice_df['user_name'] = invoice_df['user_name'].str.strip()

rate_card = invoice_df.drop_duplicates(
    subset=['user_name', 'project_name', 'hourly_rate'])[[
        'user_name', 'project_name', 'hourly_rate'
    ]]
rate_card = rate_card.dropna(axis=0,
                             subset=['project_name', 'user_name']).sort_values(
                                 ['project_name', 'user_name']).reset_index()
rate_card.drop(['index'], 1, inplace=True)
# dfmi.loc[:, ('one', 'second')]
rate_card.loc[rate_card.user_name == 'Sunil Kumar Shambulingaiah', 'user_name'] = 'Sunil Shambulingaiah'

rate_cd = pd.merge(rate_card, users3, on='user_name', how='left')
# rate_cd.loc[rate_cd.user_name == 'Sunil Shambulingaiah']


## merge time entries and invoices monthly

In [22]:
invoice_df.rename(columns = {'quantity': 'hours', 'ttl_item_amount' : 'billable_amt'}, inplace =True)

merge_all.rename(columns = {'rounded_hours': 'hours'}, inplace = True)
merge_all['billable_amt'] = merge_all['hourly_rate']*merge_all['hours']

In [23]:
invoice_df['month'] = pd.to_datetime(invoice_df['issue_date']).dt.floor('d') - pd.offsets.MonthBegin(1)

describe = invoice_df['description'].str.split(':', expand = True)[1]
invoice_df['user_name'] = describe.str.split("(", expand = True)[0]
# merge_all = pd.merge(assign_merge, users3, on = 'user_id', how='left')
# rate_cd[rate_cd.BusUnit.isnull()]
invoice_df.user_name = invoice_df['user_name'].str.strip()


In [24]:
invoice_df.loc[invoice_df['user_name'] == 'Sunil Kumar Shambulingaiah', 'user_name'] = 'Sunil Shambulingaiah'

invoices = pd.merge(invoice_df, rate_cd, on = ['user_name','project_name', 'hourly_rate'], how='left')

invoice2 = invoices.assign(task_name = '', billable = True, spent_date = '')
invoice2 = invoice2[invoice2['state'] != 'draft']

timesheet = merge_all.assign(invoice_num = '' , invoice_amt = '', discount_amount = '', subject = '',\
                state = 'time_entries', issue_date = '', services_provided = '', description = '')

# Get last invoice date
max_date_inv = max(pd.to_datetime(invoice2['issue_date'])) + timedelta(days=1)
b4_this_week = today - timedelta(today.weekday()+ 1)
this_month = today.replace(day=1)
timesheet['spent_date'] = pd.to_datetime(timesheet['spent_date'] )

# only take this month's times up to current week for financial tracker
timesheet2 = timesheet.loc[(timesheet['spent_date'] >= pd.to_datetime(this_month))]
timesheet2 = timesheet2.loc[(timesheet['spent_date'] <= pd.to_datetime(b4_this_week))]

In [25]:
timesheet2 = timesheet2[[
    'client_name', 'project_name', 'project_code', 'month', 'user_name',
    'hours', 'hourly_rate', 'billable_amt', 'billable', 'role', 'BusUnit',
    'task_name', 'invoice_num', 'invoice_amt', 'discount_amount', 'subject',
    'state', 'issue_date', 'services_provided', 'description'
]]

invoice2 = invoice2[[
    'client_name', 'project_name', 'project_code', 'month', 'user_name',
    'hours', 'hourly_rate', 'billable_amt', 'billable', 'role', 'BusUnit',
    'task_name', 'invoice_num', 'invoice_amt', 'discount_amount', 'subject',
    'state', 'issue_date', 'services_provided', 'description'
]]
invoice2.user_name = invoice2.loc[:, 'user_name'].str.strip()
time_entries = pd.concat([timesheet2, invoice2])

# missing items check
# time_entries.loc[time_entries.BusUnit.isnull()& time_entries.user_name.isnull(),'description'].unique()
# users3[users3['BusUnit'].isnull()]

time_entries.loc[time_entries.billable == False, 'billable_amt'] = 0

time_entries.to_excel(r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\time_entries.xlsx', index = False)

# time_entries.loc[(time_entries.project_name == 'CCO Automation') & (time_entries.month == '2021-12-01')]

In [26]:
def Archive(name):
    path = r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData'
    dt = str(datetime.datetime.now().date()).replace('-','_')
    og = path  + '\\' + name + '.xlsx'
    new = path + '\\Archive\\' + name + dt + '.xlsx'
    shutil.copy2(og,new)

Archive('time_entries')

## Scrape Forecast data

In [27]:
# Scrape forecast data from Harvest Forecast
config = configparser.ConfigParser()
config.read(r'C:\Users\GisselleMaira\test.ini')

user = config['harvest']['email']
pswd = config['harvest']['pswd']

In [28]:
driver = webdriver.Chrome()

In [29]:
time.sleep(2)
driver.get("https://id.getharvest.com/harvest/sign_in")
#driver.current_url
time.sleep(5)

In [30]:
email = driver.find_element(By.ID, "email")
pwd = driver.find_element(By.ID, "password")

In [31]:
email.click()
email.send_keys(user)

In [32]:
pwd.click()
# pwd.sleep(5)
pwd.send_keys(pswd)

In [33]:
submit = driver.find_element(By.ID, "log-in")
submit.click()

In [34]:
frcst = driver.find_element(By.CSS_SELECTOR, "#forecast-accounts > li > a")
frcst.click()
time.sleep(2)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#forecast-accounts > li > a"}
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x007BE713+2549523]
	Ordinal0 [0x007572D1+2126545]
	Ordinal0 [0x00652C88+1059976]
	Ordinal0 [0x0067E24E+1237582]
	Ordinal0 [0x0067E44B+1238091]
	Ordinal0 [0x006A9002+1413122]
	Ordinal0 [0x00697914+1341716]
	Ordinal0 [0x006A73AA+1405866]
	Ordinal0 [0x00697736+1341238]
	Ordinal0 [0x006734E6+1193190]
	Ordinal0 [0x00674376+1196918]
	GetHandleVerifier [0x00959742+1627410]
	GetHandleVerifier [0x00A0A93C+2352908]
	GetHandleVerifier [0x00852AE1+551089]
	GetHandleVerifier [0x00851B03+547027]
	Ordinal0 [0x0075CBEE+2149358]
	Ordinal0 [0x00760F18+2166552]
	Ordinal0 [0x00761060+2166880]
	Ordinal0 [0x0076AC60+2206816]
	BaseThreadInitThunk [0x7734FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77497A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77497A6E+238]


In [ ]:
# move to reports page
export = 'https://forecastapp.com/1508320/export'
driver.get(export)
time.sleep(2)

In [ ]:
#select time frame
tm_rng = driver.find_element(By.CSS_SELECTOR, ".input-appearance.ember-view")
tm_rng.click()

# start with weekly view of this month
tm_rng.send_keys(Keys.ARROW_DOWN) 

tm_rng.click()

In [ ]:
# download
# driver.find_element_by_class_name("button").click()

driver.find_element(By.CLASS_NAME, "button").click()

In [ ]:
folder_path = r'C:\Users\GisselleMaira\Downloads'
file_type = '\*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)
#print (max_file)

folder_path2 = r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData'
proj_file = folder_path2 + '\\project_weekly.csv'

shutil.move(max_file, proj_file)


In [ ]:
# switch to monthly
driver.find_element(By.CSS_SELECTOR, "div:nth-child(4) > div > label:nth-child(2)").click()

In [ ]:
#select time frame
tm_rng = driver.find_element(By.CSS_SELECTOR, ".input-appearance.ember-view")
tm_rng.click()


In [ ]:
# select 6 months
tm_rng.send_keys(Keys.ARROW_DOWN) # 3 month
tm_rng.send_keys(Keys.ARROW_DOWN) # 6 month
# tm_rng.send_keys(Keys.ARROW_DOWN) #this year
# tm_rng.send_keys(Keys.ARROW_DOWN) # Custom
tm_rng.click()

# download
# driver.find_element_by_class_name("button").click()
driver.find_element(By.CLASS_NAME, "button").click()

In [ ]:
folder_path = r'C:\Users\GisselleMaira\Downloads'
file_type = '\*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)
#print (max_file)

# folder_path2 = r'C:\Users\GisselleMaira\Desktop'
proj_file = folder_path2 + '\\project_monthly.csv'

shutil.move(max_file, proj_file)

In [ ]:
# switch back to weekly
driver.find_element(By.CSS_SELECTOR, "div:nth-child(4) > div > label:nth-child(1)").click()

In [ ]:
#select time frame
tm_rng = driver.find_element(By.CSS_SELECTOR, ".input-appearance.ember-view")
tm_rng.click()

tm_rng.send_keys(Keys.ARROW_UP) 
tm_rng.click()

# download
# driver.find_element_by_class_name("button").click()
driver.find_element(By.CLASS_NAME, "button").click()

In [ ]:
folder_path = r'C:\Users\GisselleMaira\Downloads'
file_type = '\*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)
#print (max_file)

# folder_path2 = r'C:\Users\GisselleMaira\Desktop'
proj_file = folder_path2 + '\\project_weekly2.csv'

shutil.move(max_file, proj_file)

In [ ]:
driver.quit()

## Wrangle the forecast data to merge

In [ ]:
proj_wk = pd.read_csv (r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\project_weekly2.csv')

col_to_move1 = proj_wk.columns.tolist()[7:]
projectW_ = pd.melt(proj_wk, id_vars=['Client', 'Project', 'Person', 'Roles'], value_vars= col_to_move1, value_name='Hours')

projectW_.rename(columns= {'variable': 'month', 'Hours': 'Forecast', 'Client': 'client_name',\
                         'Project':'project_name', 'Person': 'user_name'}, inplace=True)

In [ ]:
# import datetime
today = date.today()
this_wk  = today + datetime.timedelta(days=-today.weekday())

# get future weeks of current month
projectW_['month'] = pd.to_datetime(projectW_['month']).dt.date
projectW_ = projectW_.loc[(projectW_.month >= this_wk)]

#### weekly forecast

In [ ]:
projectW_.rename(columns = {'month':'week'}, inplace=True)
projectW_['month'] = this_wk.replace(day=1)

This_month = this_wk.replace(day=1)

In [ ]:
proj_wk2 = pd.read_csv(r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\project_weekly2.csv')

col_to_move1 = proj_wk2.columns.tolist()[7:]
projectW2_ = pd.melt(proj_wk2,
                     id_vars=['Client', 'Project', 'Person', 'Roles'],
                     value_vars=col_to_move1,
                     value_name='Hours')

projectW2_.rename(columns= {'variable': 'week', 'Hours': 'Forecast', 'Client': 'client_name',\
                         'Project':'project_name', 'Person': 'user_name'}, inplace=True)

In [ ]:
projectW2_['Roles'] = projectW2_['Roles'].replace(
    ['Consultant, Data and Analytics, Flying Phase'],
    'Consultant, Data and Analytics')

projectW2_.loc[projectW2_.user_name == 'Stephanie Lennon',
               'Roles'] = 'Principal, Data and Analytics'

projectW2_['role'] = projectW2_['Roles'].str.split(',', expand=True)[0]
projectW2_['BusUnit'] = projectW2_['Roles'].str.split(',', expand=True)[1]
# projectW2_.drop('Roles', 1, inplace=True)

rates = time_entries.drop_duplicates([
    'project_name', 'project_code', 'client_name', 'user_name', 'hourly_rate', 'billable'
])[['project_name', 'project_code', 'client_name','user_name', 'hourly_rate', 'billable']]

In [ ]:
forecast3 = pd.merge(projectW2_,
                     rates,
                     on=['project_name', 'user_name'],
                     how='left')

forecast3.rename(columns={'client_name_x': 'client_name'}, inplace=True)
# forecast3.head(2)
rates_stand = rates.loc[
    rates.billable == True,
    ['user_name', 'client_name', 'hourly_rate']].sort_values('user_name')

rates_stand = rates_stand.loc[rates_stand.user_name.notnull()].groupby(
    'user_name').first()

In [ ]:
# forecast3.loc[forecast3.billable.isnull()]
mask1 = forecast3.client_name != '0000_Spinnaker'
mask2 = forecast3.billable.isnull()

missing_rates = forecast3.loc[mask1 & mask2].drop_duplicates(
    ['user_name', 'project_name',
     'client_name'])[['user_name', 'project_name', 'client_name']]

missing_rates = pd.merge(missing_rates,
                         rates_stand,
                         on=['user_name', 'client_name'],
                         how='left')

rates2 = rates.loc[:,
                   ['client_name', 'project_name', 'project_code', 'billable'
                    ]].drop_duplicates()

missing_rates2 = pd.merge(missing_rates,
                          rates2,
                          on=['client_name', 'project_name'],
                          how='left')

In [ ]:
# forecast4 is at weekly level
forecast4 = pd.merge(forecast3,
                     missing_rates2,
                     on=['user_name', 'project_name', 'client_name'],
                     how='left')
# forecast4.loc[forecast4.user_name == 'Kevin Baquero']

forecast4['billable'] = forecast4['billable_x'].fillna(forecast4.billable_y)

forecast4 = forecast4.assign(
    project_code=forecast4.project_code_x.fillna(forecast4.project_code_y),
    hourly_rate=forecast4.hourly_rate_x.fillna(forecast4.hourly_rate_y),
    billable=forecast4.billable_x.fillna(forecast4.billable_y))

forecast4.loc[forecast4.client_name == '0000_Spinnaker', 'billable'] = False

forecast4 = forecast4.assign(month='',
                 hours='',
                 billable_amt=forecast4.Forecast * forecast4.hourly_rate,
                 task_name='',
                 invoice_num='',
                 invoice_amt='',
                 discount_amount='',
                 subject='',
                 state='forecast',
                 issue_date='',
                 services_provided='',
                 description='')

forecast4 = forecast4[[
    'client_name', 'project_name', 'project_code', 'week', 'user_name',
    'Forecast', 'hourly_rate', 'billable', 'role', 'BusUnit', 'task_name',
    'invoice_num', 'invoice_amt', 'discount_amount', 'subject', 'state',
    'issue_date', 'services_provided', 'description'
]]

#### Flat Fees Weekly

In [ ]:
# add rows for monthly flat fees with anon user_name and remove other billable amts
# this is only for current projects with Flat Fee structure
FlatFee = pd.read_excel (r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\FlatFees.xlsx')

# create monthly days and combine
full_cal = pd.read_excel (r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\Calendar.xlsx')
cal_month = full_cal.groupby('Month')['days'].sum()
cal_week = full_cal.groupby('Wk_start')['days'].sum()
full_cal = full_cal.set_index('dates')
# dates2 = pd.merge(dates, cal_month, left_on='Start', right_on ='Month', how='left')

Fees2 = pd.merge(FlatFee,cal_month, left_on= 'month' , right_on = 'Month', how='left')
Fees2 = Fees2.assign(daily = Fees2.billable_amt /Fees2.days )[['month', 'daily', 'days', 'project_name', 'client_name', 'BusUnit']]

FF_projs = list(set(FlatFee['project_name']))
FF_projs2 = pd.DataFrame(list(enumerate(FF_projs,1)))
FF_projs3 = "FF" +  FF_projs2.astype(str)  



In [ ]:
# get weeks broken by month days
Fees3 = pd.merge(full_cal.groupby(['Wk_start', 'Month'],
                                  as_index=False)['days'].sum(),
                 Fees2,
                 left_on='Month',
                 right_on='month',
                 suffixes=['_wkly', '_mnthly'])

Fees3 = Fees3.assign(weekly=Fees3.daily * Fees3.days_wkly).drop(
    ['Month', 'daily', 'days_mnthly'], 1)

In [ ]:
Fees4 = Fees3[['client_name', 'project_name', 'Wk_start', 'weekly',
               'month', 'BusUnit']].rename({
                   'Wk_start': 'week',
                   'weekly': 'billable_amt'
               },
                                axis=1)
# Fees3
Fees4 = Fees4.assign(billable=True, state='projection')

Fees4['week'] = pd.to_datetime(Fees4['week']).dt.date

FeesW = pd.merge(Fees4,
                 FlatFee.head(1)[['project_name', 'project_code']],
                 on='project_name',
                 how='left')

In [ ]:
# timesheet['week'] = timesheet['spent_date'].dt.to_period('W').apply(lambda x: x.start_time)
timesheetW = timesheet.groupby([
    'user_name', 'client_name', 'project_name', 'project_code', 'hourly_rate',
    'role', 'BusUnit', 'wk_start', 'billable_amt', 'state'
]).sum().reset_index()[[
    'user_name', 'client_name', 'project_name', 'project_code', 'hourly_rate',
    'role', 'BusUnit', 'wk_start', 'billable_amt', 'state', 'hours'
]]

# get billable tags by project for missing items
projs = timesheet.drop_duplicates(['project_name',
                           'billable'])[['project_name', 'billable'
                                         ]].sort_values('project_name')

timesheetW = pd.merge(timesheetW, projs, on='project_name', how='left')

In [ ]:
FeesW = FeesW.groupby([
    'client_name', 'project_name', 'project_code', 'week', 'billable', 'state', 'BusUnit'
]).sum().reset_index()
FeesW['week'] = pd.to_datetime(FeesW['week'])
Fees_W = FeesW.rename({'week': 'wk_start'}, axis=1)

# combine time entries with projected projects
weekly_time = pd.concat([Fees_W, timesheetW], axis=0)

# remove billable_amt for flat fee projects
weekly_time.loc[ (weekly_time.state == 'time_entries') &  (weekly_time.project_name.isin(FF_projs)), 'billable_amt'] = 0



In [ ]:
forecast5 = forecast4[[
    'client_name', 'project_name', 'week', 'billable', 'state', 'project_code',
    'user_name', 'hourly_rate', 'role', 'BusUnit', 'Forecast'
]].rename({'week': 'wk_start'}, axis=1)

forecast5['billable_amt'] = np.where(
    forecast5.project_name.isin(FF_projs), 0,
    forecast5.Forecast * forecast5.hourly_rate)
forecastW = pd.concat([weekly_time, forecast5], axis=0)

forecastW['T_hours'] = forecastW['Forecast'].fillna(0) + forecastW['hours'].fillna(0)

In [ ]:
forecastW2 = forecastW.loc[forecastW.billable == True, [
    'project_name', 'wk_start', 'state', 'role', 'BusUnit', 'billable_amt'
]]

# forecastW2.groupby(['project_name', 'wk_start', 'state', 'role', 'BusUnit']).sum().reset_index()

forecastW2['wk_start'] = pd.to_datetime(forecastW2['wk_start'])
forecastW2.set_index('wk_start')

forecastW3 = forecastW2.loc[forecastW2.wk_start >= '2022-01-03'].groupby(['wk_start']).sum().round(2)



In [ ]:
wk_proj = pd.concat([forecastW3, forecastW3.cumsum(axis=0).rename({'billable_amt':'cume'}, axis=1)], axis=1)

wk_proj = wk_proj.assign(remaining = 2500000 - wk_proj['cume']).reset_index()

end_date = datetime.date(2022, 6, 30)
end_date_wk= (end_date - datetime.timedelta(days=end_date.weekday()))

def diff(start, end):
    x = pd.to_datetime(end) - pd.to_datetime(start)
    return int(x / np.timedelta64(1, 'W'))

wk_proj['weeks_left'] = wk_proj['wk_start'].apply(lambda x: diff(x,end_date_wk))
wk_proj = wk_proj.assign(needed = wk_proj['remaining'] / wk_proj['weeks_left'])
wk_proj = wk_proj.assign(wk_new = wk_proj.wk_start.dt.date)

In [ ]:
needed = wk_proj.loc[wk_proj.wk_new == this_wk, 'needed']

after_curr = [x >= this_wk for x in wk_proj.wk_new]

wk_proj['wk_goal'] = np.where(after_curr, int(needed), 0)

wk_proj2 = wk_proj[['wk_start', 'billable_amt', 'cume', 'wk_goal']]

wk_proj2.to_excel(
    r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\weekly.xlsx',
    index=False)
Archive('weekly')

#### Monthly forecast

In [ ]:
proj_mnth = pd.read_csv(
    r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\project_monthly.csv'
)
col_to_move2 = proj_mnth.columns.tolist()[7:]

projectM_ = pd.melt(proj_mnth,
                    id_vars=['Client', 'Project', 'Person', 'Roles'],
                    
                    value_vars=col_to_move2,
                    value_name='Hours')
projectM_ = projectM_[projectM_['Hours'] > 0]

projectM_.rename(columns= {'variable': 'month', 'Hours': 'Forecast', 'Client': 'client_name',\
                         'Project':'project_name', 'Person': 'user_name'}, inplace=True)

In [ ]:
# Create forecast with future months and future weeks in current month
This_month = pd.to_datetime(This_month)

projectM_.month = pd.to_datetime(projectM_.month)
projectW_.month = pd.to_datetime(projectW_.month)

projectM_ = projectM_.loc[(projectM_['month'] != This_month)]
# projectW_.drop(['week'],1, inplace=True)

forecast = pd.concat([projectW_,projectM_], axis=0, ignore_index=True)

In [ ]:
forecast['Roles'] = forecast['Roles'].replace(
    ['Consultant, Data and Analytics, Flying Phase'],
    'Consultant, Data and Analytics')

forecast.loc[forecast.user_name == 'Stephanie Lennon', 'Roles'] = 'Principal, Data and Analytics'

forecast['role'] = forecast['Roles'].str.split(',', expand=True)[0]
forecast['BusUnit'] = forecast['Roles'].str.split(',', expand=True)[1]
forecast.drop('Roles', 1, inplace=True)

In [ ]:
# monthly needed for specific reports
forecast2 = pd.merge(forecast, rates, on=['project_name', 'user_name', 'client_name'])
forecast2['billable_amt'] = forecast2.hourly_rate * forecast2.Forecast

forecast2 = forecast2.assign(hours=0,
                             task_name='',
                             invoice_num='',
                             invoice_amt='',
                             discount_amount='',
                             subject='',
                             state='forecast',
                             issue_date='',
                             services_provided='',
                             description='')

In [ ]:
# Fees2 provides flat fee revenue structure by month, remove past month(s)
FFprojects = Fees2.project_name.unique()

# eliminate billable amt for fixed fee projects
forecast2.loc[forecast2.project_name.isin(FFprojects), 'billable_amt'] = 0
forecast2 = forecast2.assign(FF = np.where(forecast2.project_name.isin(FFprojects),1,0))

In [ ]:
filter_dt = '2022-' + str(datetime.datetime.today().month) + '-1'

Fees5 = Fees2.assign(billable=True,
                     billable_amt=Fees2['daily'] * Fees2['days'],
                     state='projection').drop(['daily', 'days'], axis=1)

In [ ]:
Fees5 = Fees5.loc[Fees5.month >= filter_dt]

# merge fixed fee with 
forecastM = pd.concat([Fees5, forecast2], axis=0)

In [ ]:
#  Get all invoices and first project_name to fill in gaps
names = time_entries.drop_duplicates(subset='invoice_num')[['invoice_num', 'project_name', 'project_code']]
time_entries_merge = pd.merge(time_entries, names, on='invoice_num', how='left')

time_entries_merge = time_entries_merge.assign(
    project_name=np.where(time_entries_merge.project_name_x.isnull(),
                          time_entries_merge.project_name_y,
                          time_entries_merge.project_name_x),
    project_code=np.where(time_entries_merge.project_code_x.isnull(),
                          time_entries_merge.project_code_y,
                          time_entries_merge.project_code_x)).drop([
                              'project_name_y', 'project_code_y',
                              'project_name_x', 'project_code_x'
                          ], 1)

time_entries_merge['user_name'] = time_entries_merge['user_name'].str.strip()
time_entries_fix = pd.merge(time_entries_merge,
                            rate_card,
                            on=['user_name', 'project_name', 'hourly_rate'],
                            how='left')


In [ ]:
all = pd.concat([time_entries_fix, forecastM])[[ 'client_name', 'project_name', 'project_code', 'month', 'hours',
    'hourly_rate', 'billable_amt', 'billable', 'role', 'BusUnit',
    'invoice_num', 'invoice_amt', 'state', 'description']]

# alltime_entries.fillna({'Forecast':0, 'hours':0, 'FF': 0}, inplace=True)
# monthly = alltime_entries[[
#     'client_name', 'project_name', 'project_code', 'month', 'hours',
#     'hourly_rate', 'billable_amt', 'billable', 'role', 'BusUnit',
#     'invoice_num', 'invoice_amt', 'state', 'description', 'Forecast', 'FF'
# ]]

all.to_excel(
    r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\monthly.xlsx',
    index=False)

Archive('monthly')


## CRM refresh or Sharepoint

In [ ]:
option = Options()

option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

# Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2 
})

driver = webdriver.Chrome(options=option)
# driver = webdriver.Chrome(chrome_options=option, executable_path='path-of-driver\chromedriver.exe')

In [ ]:
# Scrape forecast data from Harvest Forecast
config = configparser.ConfigParser()
config.read(r'C:\Users\GisselleMaira\test.ini')

user = config['Zoho']['email2']
pswd = config['Zoho']['pswd2']

In [ ]:
time.sleep(2)
driver.get("https://accounts.zoho.com/signin")

time.sleep(2)

email = driver.find_element(By.ID, "login_id")
email.click()
email.send_keys(user)

driver.find_element(By.ID, "nextbtn").click()
time.sleep(2)

pwd = driver.find_element(By.ID, "password")
pwd.send_keys(pswd)

driver.find_element(By.ID, "nextbtn").click()
time.sleep(2)

driver.get('https://crm.zoho.com/crm/org771208238/settings/export-data')
time.sleep(2)

In [ ]:
time.sleep(2)

sel = Select(driver.find_element_by_xpath("//select[@name='module']"))
sel.select_by_visible_text('Deals')

In [ ]:
driver.find_element(By.ID, "Submit").click()
driver.switch_to.alert.accept()
time.sleep(2)

In [ ]:
driver.quit()

In [ ]:
def move_file():
    ''' moves last downloaded file to saved subfolder'''
    folder_path = r'C:\Users\GisselleMaira\Downloads'
    file_type = '\*csv'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)
    shutil.move(max_file, proj_file)

In [ ]:
folder_path2 = r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData'
proj_file = folder_path2 + '\\ZohoDeals.csv'

move_file()

## New Opps (weekly)


In [ ]:
# import datetime
deals = pd.read_csv (r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\ZohoDeals.csv')

today = date.today()
latest_wk = today - timedelta(today.weekday()+ 1)
prior_wk = latest_wk - datetime.timedelta(days=7)

In [ ]:
deals = deals.rename({'Probability (%)': 'Prob'}, axis=1)[[
    'Deal Owner', 'Amount', 'Expected Revenue', 'Deal Name', 'Account Name',
    'Stage', 'Created Time', 'Modified Time', 'Proposal Type',
    'Expected Start Date', 'Expected End Date', 'Nature of Work', 'Prob'
]]

cols = [
    'Created Time', 'Expected Start Date', 'Expected End Date', 'Modified Time'
]
deals[cols] = deals[cols].apply(pd.to_datetime)

deals.rename(
    {
        'Expected Start Date': 'start',
        'Expected End Date': 'end',
        'Created Time': 'Created',
        'Modified Time': 'Modified'
    },
    axis=1,
    inplace=True)

In [ ]:
Won = deals.loc[(deals.Prob > 50)
                & (deals.Stage == 'Closed Won - Land & Expand') &
                (deals.start.notnull()) & (deals.start > filter_dt)]

Pending = deals.loc[(deals.Prob > 50) & (~deals['Stage'].str.contains('Closed')) &
          (deals.start.notnull()) & (deals.start > filter_dt)]

all_projs = pd.concat([Won, Pending], axis=0)

In [ ]:
Won

In [ ]:
Pending

In [ ]:
# create generic table for dates
today = date.today()
tgt = datetime.date(2022, 1, 1)
next_6month = today + relativedelta(months=7)

# # start/end dates for each month
M_start = pd.date_range(tgt, next_6month,freq='MS')
M_end = pd.date_range(tgt, next_6month,freq='M')
dates = pd.DataFrame(list(zip(M_start, M_end)), columns =['Start', 'End'])
dates['Month'] = dates['Start'].dt.strftime('%Y-%m')

# create monthly days and combine
full_cal = pd.read_excel (r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\Calendar.xlsx')
cal_month = full_cal.groupby('Month')['days'].sum()
dates2 = pd.merge(dates, cal_month, left_on='Start', right_on ='Month', how='left')

full_cal = full_cal.set_index('dates')


In [ ]:
# replace empty dates with placeholders 
# nodate = datetime.datetime(2099, 1, 1)
# nodate2 = datetime.datetime(2099, 1, 1)

# Probable['start'].fillna(nodate, inplace = True)
# Probable['end'].fillna(nodate2, inplace = True)

# create holiday file
us_hol_date = []
us_hol_name = []

comp_holidays = ["New Year's Day",
                 "Martin Luther King Jr. Day",
                 "Washington's Birthday",
                 'Memorial Day',
                 'Independence Day',
                 'Labor Day', 
                 'Thanksgiving', 
                 'Christmas Day']
for day, name in holidays.US(years=[2022, 2023], observed=True).items():
    if name in comp_holidays:
        us_hol_name.append(name)
        us_hol_date.append(day)
maskcal=np.busdaycalendar(holidays=us_hol_date)



In [ ]:
# calc busdays and daily revs
all_projs = all_projs.assign(
    diff_days=np.busday_count(all_projs['start'].values.astype('datetime64[D]'),
                              all_projs['end'].values.astype('datetime64[D]'),
                              busdaycal=maskcal))

all_projs = all_projs.assign(daily = all_projs['Expected Revenue']/all_projs['diff_days'])

all_projs = all_projs.loc[all_projs.start > pd.to_datetime(today)]

# all_projs
# NewRevs = Probable.loc[Probable.diff_days > 0]

In [ ]:
# create dataframes per project and combine
newP = []
for i in range(len(all_projs)):
    new = dates2.loc[(all_projs['start'].iloc[i] <= dates2.End)
                     & (all_projs['end'].iloc[i] >= dates2.Start)].assign(
                         project=all_projs['Deal Name'].iloc[i],
                         owner=all_projs['Deal Owner'].iloc[i],
                         Client=all_projs['Account Name'].iloc[i],
                         stage=all_projs['Stage'].iloc[i],
                         daily=all_projs['daily'].iloc[i],
                         start=all_projs['start'].iloc[i],
                         end=all_projs['end'].iloc[i])
    newP.append(new)
new = pd.concat(newP)

# replace start/end dates with project start/end dates as approp
new = new.assign(
    NewStart=np.where(new['Start'] > new['start'], new['Start'], new['start']),
    NewEnd=np.where(new['End'] < new['end'], new['End'],
                    new['end'])).drop(['Start', 'End', 'start', 'end', 'days'], axis=1)

# check for in progress projects
# Last_fri = date.today() + timedelta(days=-today.weekday()) + timedelta(days=4)
# new.loc[new.NewStart < pd.to_datetime(today), 'NewStart'] = Last_fri
# all.loc[all.client_name.str.contains('Hard'),['project_name', 'state', 'month']]

# get busdays
new = new.assign(
    diff_days=np.busday_count(new['NewStart'].values.astype('datetime64[D]'),
                              new['NewEnd'].values.astype('datetime64[D]'),
                              busdaycal=maskcal))

new = new.assign(Revenue = new.diff_days * new.daily)

In [ ]:
new2 = new.rename(columns={
    'Client': 'client_name',
    'project': 'project_name',
    'Revenue': 'billable_amt'
}).assign(billable=True,
          state='NewProject',
          project_code='Prospective Project').drop(
              ['daily', 'NewStart', 'NewEnd', 'diff_days'], axis=1)

# convert mo-yr to trunc date
new2.Month = pd.to_datetime(new2.Month)
new2 = new2.rename(columns={'Month': 'month'})

new2['client_name'] = 'ZProspective_' + new2['client_name'].astype(str)

In [ ]:
all = all.assign(Type = 'Current')
new2 = new2.assign(Type='New')

final = pd.concat([all, new2], axis=0)
final2 = final.drop_duplicates()
final2.loc[(final2.project_name.isin(FF_projs)) & (final2.state == 'time_entries'), 'billable_amt'] = 0
# final2.loc[(final2.project_name.isin(FF_projs)) & (final2.state == 'time_entries')]
final2.loc[(final2.project_name.isin(FF_projs)) & (final2.month == '2022-03-01') & (final2.state == 'projection'), 'billable_amt'] = 0 


final2.BusUnit = final2.BusUnit.str.strip()

In [ ]:
final3 = final2.assign(order=np.where(final2.Type == 'Current', 1, 4))

CurrProj = final3.loc[final3.Type == 'Current'].groupby(['month']).sum()[[
    'billable_amt'
]].assign(order=2, project_name='All Current Projects').reset_index()

CurrProj_noFP = final3.loc[
    (final3.Type == 'Current')
    & (final3.BusUnit != 'Flying Phase')].groupby(['month']).sum()[[
        'billable_amt'
    ]].assign(order=2, project_name='All Current Projects w/o FP').reset_index()

subtotal_New = final3.loc[final3.Type == 'New'].groupby([
    'month', 'Type'
]).sum()[['billable_amt']].assign(order=5,
                                  project_name='All Prospective Projects',
                                  state='all').reset_index()

In [ ]:
month_ttls = final3.groupby('month').sum()[['billable_amt']].assign(
    project_name='All Projects', state='all', Type='all',
    order=6).reset_index()

month_ttls_woFP = final3.loc[final3.BusUnit != 'Flying Phase'].groupby('month').sum()[[
    'billable_amt'
]].assign(project_name='All Projects w/o FP', state='all', Type = 'all',
          order=7).reset_index()

In [ ]:
totals = pd.concat([CurrProjAll, CurrProj_noFP, subtotal_New, month_ttls, month_ttls_woFP])
final4 = pd.concat([final3, totals])
final4 = final4.drop_duplicates()

# pd.to_datetime(totals['month']).dt.floor('d')

In [ ]:
final4.to_excel(r"C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\forecast3.xlsx", index=False) 

In [ ]:
# ttl_join = pd.merge(
#     month_ttls.drop(['BusUnit', 'state', 'order', 'project_name'], 1),
#     FP_ttls.drop(['BusUnit', 'state', 'order', 'project_name'], 1),
#     on='month')

# ttl_join = ttl_join.assign(billable_amt=ttl_join.billable_amt_x -
#                            ttl_join.billable_amt_y,
#                            state='total',
#                            order=4,
#                            project_name='All Projects excl FP').drop(
#                                ['billable_amt_x', 'billable_amt_y'], 1)

# final2 = final2.assign(order = 1)

# final3 = pd.concat([final2, month_ttls, FP_ttls, ttl_join], axis=0)

# final3 = final3[['order', 'client_name', 'project_name', 'project_code', 'month', 'hours',
#        'hourly_rate', 'billable_amt', 'billable', 'role', 'BusUnit',
#        'invoice_num', 'invoice_amt', 'state', 'description', 'owner', 'stage']]

# final3.to_excel(r"C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\forecast2.xlsx", index=False) 

# final3[['order', 'client_name', 'project_name', 'project_code', 'month', 'hours',
#        'hourly_rate', 'billable_amt', 'billable', 'role', 'BusUnit',
#        'invoice_num', 'invoice_amt', 'state', 'description', 'owner', 'stage']]

In [ ]:
# Prospects['Expected Start Date']

# proj_len = []
# for i in range(len(Prospects)):
#     proj_len.append(full_cal.loc[
#         (full_cal.index >= pd.DatetimeIndex(Prospects['Expected Start Date'])[i]) ,
# #         &
# #         (full_cal.index <= pd.DatetimeIndex(Prospects['Project End'])[i]),
#         'days'].sum())    

# rpt['daily'] = pd.to_numeric(rpt['Project Budget'].str.replace(',', ''),
#                              errors='coerce') / proj_len

# fnl_forecast = pd.concat([alltime_entries,prospects], axis=0)

# fnl_forecast.to_excel(r"C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\forecast.xlsx", index=False) 

## Emp Counts

In [ ]:
# pull start/end (term) dates from file and generate datelist
edates = pd.read_excel(
    r'C:\Users\GisselleMaira\Desktop\SLT Docs\SeniorityReport.xlsx')

edates['user_name'] = edates['Preferred/First Name'] + ' ' + edates['Last Name']
edates.drop([
    'Last Name', 'Preferred/First Name', 'Tenure (In English)',
    'Tenure (in Months)'
],
           1,
           inplace=True)
active = datetime.datetime(2025, 1, 1)
edates["Termination Date"].fillna(active, inplace = True)


#generate dataset for 18 weeks 
today = date.today()
this_wk = today + timedelta(days=-today.weekday(), weeks=0)
start = today + timedelta(days=-today.weekday(), weeks=-16)

# empty date range
datelist = pd.date_range(start=start, periods=18, freq='W-MON').tolist()
# datelist2 = pd.date_range(start = start, periods = 13, freq='W-SUN').tolist()
newdates = [d.strftime('%m-%d-%Y') for d in datelist]

In [ ]:
# cross join sets and add active indicators
newdates = pd.DataFrame(newdates, columns=['date'])
newdates['key'] = 1
edates['key'] = 1

edates2 = pd.merge(edates, newdates, on='key',
                   how='outer').drop('key', 1).rename(columns={
                       'Termination Date': 'end',
                       'Hire Date': 'Hire'
                   })

edates2['date'] = pd.to_datetime(edates2['date'])
# edates2.dtypes

edates2 = edates2.assign(
    Active=np.where((edates2['Hire'] <= edates2['date'])
                    & (edates2['end'] >= edates2['date']), 1, 0))

In [ ]:
# get role/busUnit from API data
users_3 = users3.copy()

# drop Flying Phase from dual BusUnit consultants
ind = users_3.loc[
    (users_3['user_name'].isin(['Parker Barouch', 'Kevin Baquero']))
    & (users_3['BusUnit'] == 'Flying Phase')].index

users_3.drop(ind, inplace=True)

users_3 = users_3.drop_duplicates()

empcounts = pd.merge(edates2, users_3, on='user_name', how='left')
empcounts2 = empcounts.loc[empcounts.Active == 1,
              ['user_name', 'role', 'BusUnit', 'date', 'Active']].rename(
                  columns={'date': 'wk_start'})

In [ ]:
# merge with time entries l4M
time_2 = assign_merge.copy()

time_2['spent_date'] = pd.to_datetime(time_2['spent_date']) 
time_2['wk_start'] = time_2['spent_date'] - pd.TimedeltaIndex(time_2.spent_date.dt.dayofweek,unit='d') 
time_2 = time_2[['wk_start', 'user_name','billable', 'rounded_hours', 'project_name','task_name']]


In [ ]:
times_bill = time_2.loc[time_2.billable == True].groupby(
    ['wk_start', 'user_name', 'billable', 'project_name',
     'task_name']).sum().reset_index().drop(['billable', 'task_name'], 1)

times_bill = times_bill.loc[(times_bill.wk_start < pd.to_datetime(this_wk))
                            & (times_bill.wk_start >= pd.to_datetime(start))]

In [ ]:
times_bill.to_excel( r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\empcount_billed.xlsx', index = False)

In [ ]:
# create summary tables
hrs = time_2.groupby(['wk_start', 'user_name', 'billable']).sum().reset_index()
hrs = hrs.assign(Type = np.where(hrs.billable == True, 'Bill Hrs', 'Nonbill Hrs')).drop('billable',1)

hrs2 = hrs.pivot(index=['wk_start', 'user_name'], columns = 'Type',values='rounded_hours' ).reset_index()
hrs2 = hrs2.fillna({'Bill Hrs':np.nan, 'Nonbill Hrs':np.nan}).fillna(0)
hrs2 = hrs2.assign(Total = hrs2['Bill Hrs']+ hrs2['Nonbill Hrs'])
hrs2 = hrs2.loc[(hrs2.wk_start < pd.to_datetime(this_wk)) & (hrs2.wk_start >= pd.to_datetime(start)) ]

active_hrs = pd.merge(empcounts2, hrs2, on=['user_name', 'wk_start'],  how='left')
# active_hrs.loc[(active_hrs.task_name.isnull()) & (active_hrs.Total.isnull()), 'task_name'] = 'None'
active_hrs.loc[(active_hrs.Total.isnull()), 'Total'] = 0

In [ ]:
# removes users with no hour or BusUnit
active_hrs = active_hrs.loc[(~(active_hrs.Total.isnull()) &
                             (active_hrs.BusUnit.isnull())) |
                            (active_hrs.BusUnit.notna())]
active_hrs.to_excel( r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\empcount.xlsx', index = False)

In [ ]:
# create detailed view
task_hrs = time_2.groupby(['wk_start', 'user_name', 'billable', 'project_name','task_name']).sum().reset_index()

categories = pd.read_excel(r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\projects.xlsx')
task_hrs2 = pd.merge(task_hrs, categories, how='left', on=['project_name', 'task_name'])
task_hrs2.Category = np.where(task_hrs2.Category.isnull() & task_hrs2.billable == True, 'billable', task_hrs2.Category)

task_hrs3 = task_hrs2.groupby(['wk_start', 'user_name', 'Category']).sum().reset_index().drop('billable',1)

In [ ]:
# empcounts2.loc[empcounts2.user_name == 'Jason Xie']
empcounts2['key'] = 1

cats = categories.loc[categories.Category.notnull(), 'Category'].unique()
cats = np.append(cats, ['billable'], axis=0)
cats = pd.DataFrame(cats, columns=['category']).assign(key = 1)

# task_hrs3 = pd.merge(task_hrs4.drop(['Category', 'rounded_hours'], 1).drop_duplicates(),
#                         cats,
#                         on='key',
#                         how='outer').rename({'category' : 'Category'}, axis = 1).drop('key',1)

task_hrs4 = pd.merge(empcounts2, cats, on='key', how='outer').drop('key', 1).rename({'category': 'Category'}, axis=1)

task_hrs5 = pd.merge(task_hrs4, task_hrs3, on = ['user_name', 'wk_start', 'Category'], how='left')
task_hrs5['rounded_hours'] = task_hrs5['rounded_hours'].fillna(0)

task_hrs5.to_excel( r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\empcount_task.xlsx', index = False)

In [ ]:
newdates = pd.DataFrame(newdates, columns=['date'])
newdates['key'] = 1
edates['key'] = 1

edates2 = pd.merge(edates, newdates, on='key',
                   how='outer').drop('key', 1).rename(columns={
                       'Termination Date': 'end',
                       'Hire Date': 'Hire'
                   })


In [ ]:
active2 = active_hrs.loc[active_hrs.Active == 1]
active2.loc[active2.user_name.isin([
    'Paige Wolk', 'Megan Murray', 'Tristan Andrews', 'Brooke Pray',
    'Charles Chareunsy', 'Paul Blodgett', 'Shawn Sweeney'
]), 'BusUnit'] = 'Operations'



In [ ]:


projects = assign_merge[['project_name', 'task_name', 'billable']].drop_duplicates()
# projects.to_excel( r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\projects.xlsx', index = False)

categories = pd.read_excel(r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\projects.xlsx')


In [ ]:
categories.head()

In [ ]:
ActiveEmps = active_hrs[['user_name','Active', 'BusUnit' ]].groupby(['user_name', 'BusUnit']).sum()
# ActiveEmps.loc[ActiveEmps.Active > 0]

# ActiveEmps.loc[ActiveEmps.Active == 0]


In [ ]:
assign_merge.head()

In [ ]:
active_hrs.loc[active_hrs.Total == 0]

In [ ]:
active_ttl = active_hrs[['wk_start', 'bill_hrs', 'Nonbill_hrs', 'Total', 'role', 'BusUnit']]
# active_ttl = active_ttl.groupby(['wk_start', 'role', 'BusUnit']).sum().reset_index()

active1 = active_ttl.drop('bill_hrs', 1).groupby(['wk_start', 'role', 'BusUnit']).sum().reset_index()
active2 = active_ttl.drop('Nonbill_hrs', 1).groupby(['wk_start', 'role', 'BusUnit']).sum().reset_index()

active1['Hr_type'] = 'Nonbilled'
active2['Hr_type'] = 'billed'

active1.rename(columns = {'Nonbill_hrs': 'hours'}, inplace = True)
active2.rename(columns = {'bill_hrs': 'hours'}, inplace = True)

active_ttl2 = pd.concat([active1, active2], axis=0)
active_ttl2.drop('Total', 1, inplace = True)

active_ttl2.to_excel( r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\empcount2.xlsx', index = False) 

